In [ ]:
%pip install -q langchain
%pip install -q langchain-ibm
%pip install -q langchain-community
%pip install -q ibm-watsonx-ai
%pip install -q ibm-watson-machine_learning
%pip install -q chromadb
%pip install -q tiktoken
%pip install -q beautifulsoup4
%pip install -q python-dotenv


In [1]:
#Import libraries

import os

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
#Set up keys and api endpoints

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "XHfmHfEIismZnFOi-vviiM-IHbxwnbsaFbM0DA12qhuT",
    "model" : "ibm/granite-3-2-8b-instruct",
    "project_id": "02464812-b861-454c-b58c-2cb85043d848",
}

In [3]:
# Load in model and parameters

llm = WatsonxLLM(
    model_id=credentials["model"],
    url =credentials["url"],
    apikey=credentials["apikey"],
    project_id=credentials["project_id"],
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.TEMPERATURE: 0,
        GenParams.MIN_NEW_TOKENS: 5,
        GenParams.MAX_NEW_TOKENS: 250,
        GenParams.STOP_SEQUENCES: ["Human:", "Observation"]
    },   
)
    

Model 'meta-llama/llama-4-maverick-17b-128e-instruct-fp' is not supported for this environment. Supported models: ['google/flan-t5-xl', 'google/flan-t5-xxl', 'google/flan-ul2', 'ibm/granite-13b-instruct-v2', 'ibm/granite-20b-code-instruct', 'ibm/granite-20b-multilingual', 'ibm/granite-3-2-8b-instruct', 'ibm/granite-3-2b-instruct', 'ibm/granite-3-8b-instruct', 'ibm/granite-34b-code-instruct', 'ibm/granite-3b-code-instruct', 'ibm/granite-8b-code-instruct', 'ibm/granite-guardian-3-2b', 'ibm/granite-guardian-3-8b', 'ibm/granite-vision-3-2-2b', 'meta-llama/llama-2-13b-chat', 'meta-llama/llama-3-1-70b-instruct', 'meta-llama/llama-3-1-8b-instruct', 'meta-llama/llama-3-2-11b-vision-instruct', 'meta-llama/llama-3-2-1b-instruct', 'meta-llama/llama-3-2-3b-instruct', 'meta-llama/llama-3-2-90b-vision-instruct', 'meta-llama/llama-3-3-70b-instruct', 'meta-llama/llama-3-405b-instruct', 'meta-llama/llama-4-maverick-17b-128e-instruct-fp8', 'meta-llama/llama-4-scout-17b-16e-instruct', 'meta-llama/llama-guard-3-11b-vision', 'mistralai/mistral-large', 'mistralai/mixtral-8x7b-instruct-v01']


In [4]:
# Set up prompt template

template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)

In [5]:
# Set up a chain with our prompt and LLM

agent = prompt | llm  #not sure how this works 

In [6]:
#basic query

agent.invoke({"query": "What sport is played at the US Open?"}) # has general data about tennis and us open but super speceific and gets correct answer

' Do not try to create plausible-sounding but false answers.\n\nThe sport played at the US Open is Tennis.'

In [7]:
agent.invoke({"query": "What is the capital of France?"}) #also generic question that should have data for

'\n\nThe capital of France is Paris.'

In [8]:
agent.invoke({"query": "Where was the 2024 US Open Tennis Championship?"}) # does not have data for 2024 so it says "I do not know" which is correct.

' Do not invent an answer.\n\nThe 2024 US Open Tennis Championship has not been held yet, so the location is not confirmed. Please check back closer to the event for the most accurate information.'

In [9]:
agent.invoke({"query": "Where was the 2026 US Open Tennis Championship?"}) # does not have data for 2026 so it says "I do not know" which is correct.

' Do not invent an answer.\n\nThe 2026 US Open Tennis Championships have not been officially announced yet, so the location is not confirmed. Therefore, I cannot provide the accurate location for the 2026 US Open Tennis Championship.'

In [8]:
urls = [
  #Apple scab
  "https://www.rhs.org.uk/disease/apple-and-pear-scab",
  "https://www.davey.com/insect-disease-resource-center/apple-scab/",
  "https://mortonarb.org/plant-and-protect/tree-plant-care/plant-care-resources/apple-scab/",
  "https://en.wikipedia.org/wiki/Apple_scab",
  "https://extension.psu.edu/apple-disease-apple-scab",

  #Black Rot
  "https://www.umass.edu/agriculture-food-environment/vegetable/fact-sheets/brassicas-black-rot",
  "https://www.canr.msu.edu/ipm/diseases/black_rot?language_id=",

  #Cedar apple rust
  "https://content.ces.ncsu.edu/cedar-apple-rusts",
  "https://www.fs.usda.gov/wildflowers/plant-of-the-week/gymnosporangium_juniperi-virginianae.shtml",
  "https://ohioline.osu.edu/factsheet/plpath-tree-10",

  #Powdery Mildew
  "https://edis.ifas.ufl.edu/publication/PP267",
  "https://www.business.qld.gov.au/industries/farms-fishing-forestry/agriculture/biosecurity/plants/diseases/horticultural/powdery-mildew",
  "https://hgic.clemson.edu/factsheet/powdery-mildew/",
  "https://extension.colostate.edu/topic-areas/yard-garden/powdery-mildews-2-902/",

  #Cercospora leaf spot/gray leaf spot
  "https://edis.ifas.ufl.edu/publication/PP267",
  "https://content.ces.ncsu.edu/cercospora-leaf-spot-frogeye-leaf-spot-on-pepper",
  "https://www.umass.edu/agriculture-food-environment/greenhouse-floriculture/fact-sheets/leaf-spot-diseases-of-floricultural-crops-caused-by-fungi-bacteria",

  #Common rust
  "https://cals.cornell.edu/field-crops/corn/diseases-corn/common-rust",
  "https://extension.umn.edu/corn-pest-management/common-rust-corn",
  
  #Northern leaf blight
  "https://www.cropscience.bayer.us/articles/bayer/managing-northern-corn-leaf-blight",
  "https://extension.umn.edu/corn-pest-management/northern-corn-leaf-blight",
  "https://ohioline.osu.edu/factsheet/plpath-cer-10",

  #Esca (black measles)
  "https://www.farmprogress.com/grapes/lime-sulfur-seen-as-aid-for-black-measles-of-grapes",
  "https://grapes.extension.org/grapevine-measles/",

  #Leaf blight (isariopsis leaf spot)
  "https://www.sciencedirect.com/topics/agricultural-and-biological-sciences/leaf-blight",
  "https://eos.com/blog/leaf-blight/",


  #Bacterial spot
  "http://extension.msstate.edu/publications/bacterial-speck-and-bacterial-spot-tomatoes",
  "https://www.ontario.ca/page/bacterial-diseases-tomato-bacterial-spot-bacterial-speck-and-bacterial-canker",

  #Early blight
  "https://www.gardeningknowhow.com/edible/vegetables/potato/potato-early-blight-treatment.htm",
  "https://extension.umn.edu/disease-management/early-blight-tomato-and-potato",
  "https://content.ces.ncsu.edu/early-blight-of-tomato",

  #Late blight
  "https://www.gardentech.com/disease/late-blight",
  "https://extension.umn.edu/disease-management/late-blight",
  "https://www.ndsu.edu/agriculture/extension/publications/late-blight-potato",

  #Leaf scorch
  "https://www.bartlett.com/resources/diseases/bacterial-leaf-scorch",
  "https://www.pubs.ext.vt.edu/3001/3001-1433/3001-1433.html",
  "https://www.gardenia.net/disease/leaf-scorch",

  #Mold
  "https://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISEASES/botrytis.html",
  "https://extension.umn.edu/plant-diseases/gray-mold-flower-garden",
  "https://hort.extension.wisc.edu/articles/white-mold/",

  #Spider mites (two spotted spider mite)
  "https://extension.usu.edu/planthealth/ipm/notes_ag/hemp-spider-mites",
  "https://www.udel.edu/academics/colleges/canr/cooperative-extension/fact-sheets/two-spotted-spider-mites/",

  #Target spot
  "https://plantpath.ifas.ufl.edu/u-scout/tomato/target-spot.html",
  "https://www.growingproduce.com/vegetables/take-the-right-aim-to-tame-target-spot-of-tomato/",

  #Tomato mosaic virus
  "https://www.creative-diagnostics.com/blog/index.php/what-is-tomato-mosaic-virus/",
  "https://www.myefco.com/int/green-ideas/mosaico-del-pomodoro-cose-e-come-trattarlo/",

  #Tomato yellow leaf curl virus
  "https://ipm.ucanr.edu/agriculture/tomato/tomato-yellow-leaf-curl/#gsc.tab=0",
  "https://agriculture.vic.gov.au/biosecurity/plant-diseases/vegetable-diseases/tomato-yellow-leaf-curl-virus",
  "https://content.ces.ncsu.edu/tomato-yellow-leaf-curl-virus"

]


In [9]:
# Load the documents using Langchain's WebBaseLoader for the URLs and print a sample document to see how it loaded

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
docs_list[4]

Document(metadata={'source': 'https://extension.psu.edu/apple-disease-apple-scab', 'title': 'Apple Disease - Apple Scab', 'language': 'en'}, page_content='  \n\n\n\n\n\nApple Disease - Apple Scab\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n       \n\n The store will not work correctly when cookies are disabled. JavaScript seems to be disabled in your browser.  For the best experience on our site, be sure to turn on Javascript in your browser.     Skip to Content  Menu Browse by Category Close  Animals and Livestock Animals and LivestockClose    View All   Dairy  DairyClose   View All   Product Quality   Nutrition and Feeding   Health and Care   Reproduction and Genetics   Manure and Nutrient Management   Facilities and Technology   Personal Safety    Business Management    Poultry  PoultryClose   View All   Production and Management   Product Quality   Nutrition and Feeding   Health and Care   Reproduction and Genetics   Manure and Nutrient Management   Facilities and Technology   Personal Safet

In [12]:
#split the data in these documents into smaller chunks that can be processed by the LLM

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [13]:
#intialize embedding model

embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=credentials["project_id"],
)

In [14]:
# set up a vector store the documents and index them

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="agentic-rag-chroma",
    embedding=embeddings,
)

In [15]:
# set up a retriever to get the documents from the vector store

retriever = vectorstore.as_retriever()


In [16]:
# define a "tool" that the AI agent can use to get information from the vector store
# this is similar to a function that takes in a question and returns the answer
# the tool will use the retriever to get the relevant documents and then pass them to the LLM for processing

@tool
def get_IBM_US_Open_context(question: str):
    """Get context about IBM's involvement in the 2024 US Open Tennis Championship."""
    context = retriever.invoke(question)
    return context


tools = [get_IBM_US_Open_context] # list of tools that the agent can use

In [17]:
# set up system prompt for the agent (print its thought process, the tools that were used, and the final answer)

system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools: {tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

In [18]:
# establish the human prompt (print the user input followed by the intermediate steps of the agent as part of the agent's "scratchpad")

human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

In [19]:
# establish the order of our newly defined prompts in the prompt template

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

In [20]:
#finalize our prompt template by adding the tool names, descriptions, and arguments using a partial prompt template
# So instead of needing to pass these values (tools and tool_names) every time you use the prompt, you partially apply them once using the `partial` method.
# This way, every time you use the prompt, it will automatically include the tool names and descriptions.

prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

In [21]:
# Set up agent memory to store past conversation history and context to help the agent remember what has been discussed in the past and improve its responses

memory = ConversationBufferMemory()


/var/folders/7y/_2l8lhg554d78tl37zscc3wc0000gn/T/ipykernel_54600/1271133669.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [22]:
# set up agent chain (scratchpad to format the log of the agent's thought process and actions taken, memory, prompt, and LLM)

chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory
)

In [27]:
species = "apple"
condition = "black rot"
agent_executor.invoke({"input": f"For an {species} plant with {condition}, what are the symptoms and how can it be treated?"})




> Entering new AgentExecutor chain...


AI:
```
{
  "action": "Final Answer",
  "action_input": "Symptoms of black rot on an apple plant include small, dark spots on leaves that enlarge and turn brown, often with a yellow halo. The fruit may show dark, sunken lesions, and the flesh beneath may be discolored and stringy. Treatment involves cultural management, such as removing and destroying infected plant parts, practicing good sanitation, and planting resistant varieties. Chemical management, such as fungicide sprays applied at bud break and according to label directions, can also effectively manage black rot by reducing primary infection."
}
```

> Finished chain.


{'input': 'For an apple plant with black rot, what are the symptoms and how can it be treated?',
 'history': "Human: For an apple tree with apple scab, what are the symptoms and how can it be treated?\nAI: Symptoms of apple scab on an apple tree include brown lesions on foliage and fruits, which can enlarge and darken to become more or less circular. Treatment involves cultural management, such as collecting and removing fallen leaves and infected twigs, and sanitation practices like leaf litter removal and pruning, ideally done in the fall or winter before bud break. Resistant cultivars are the best way to prevent apple scab, with many species and cultivars of apple and crabapple being resistant to the scab fungus. Chemical management, such as fungicide sprays applied at bud swell and according to label directions, can also effectively manage apple scab by reducing primary infection.\nHuman: How did IBM use watsonx at the 2024 US Open Tennis Championship?\nAI: IBM did not use Watsonx 

In [28]:
species = "apple"
condition = "healthy"
agent_executor.invoke({"input": f"For an {species} plant that is healthy, recommend preventative measures to keep the plant healthy!"})



> Entering new AgentExecutor chain...

AI:
Thought: The human is asking for preventative measures to keep an apple plant healthy. The provided context does not contain information about preventative measures for apple plants. I will use the get_IBM_US_Open_context tool to search for any relevant information about IBM's involvement in the 2024 US Open Tennis Championship that might indirectly provide insights into apple plant care.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "How did IBM use technology to support the 2024 US Open Tennis Championship?"
}
```
Observation[Document(metadata={'source': 'http://extension.msstate.edu/publications/bacterial-speck-and-bacterial-spot-tomatoes', 'title': 'Bacterial Speck and Bacterial Spot in Tomatoes | Mississippi State University Extension Service', 'language': 'en', 'description': 'Bacterial speck and spot are common bacterial diseases of tomatoes in the southeastern United States.'}, page_content='July 3, 2024 \n E

{'input': 'For an apple plant that is healthy, recommend preventative measures to keep the plant healthy?',
 'history': "Human: For an apple tree with apple scab, what are the symptoms and how can it be treated?\nAI: Symptoms of apple scab on an apple tree include brown lesions on foliage and fruits, which can enlarge and darken to become more or less circular. Treatment involves cultural management, such as collecting and removing fallen leaves and infected twigs, and sanitation practices like leaf litter removal and pruning, ideally done in the fall or winter before bud break. Resistant cultivars are the best way to prevent apple scab, with many species and cultivars of apple and crabapple being resistant to the scab fungus. Chemical management, such as fungicide sprays applied at bud swell and according to label directions, can also effectively manage apple scab by reducing primary infection.\nHuman: How did IBM use watsonx at the 2024 US Open Tennis Championship?\nAI: IBM did not u

In [24]:
agent_executor.invoke(
    {"input": "How did IBM use watsonx at the 2024 US Open Tennis Championship?"}
)



> Entering new AgentExecutor chain...


AI:

Thought: The user is asking about IBM's use of Watsonx at the 2024 US Open Tennis Championship. I need to use the get_IBM_US_Open_context tool to find this information.

Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "How did IBM use Watsonx at the 2024 US Open Tennis Championship?"
}
```

Observation[Document(metadata={'language': 'en', 'source': 'http://extension.msstate.edu/publications/bacterial-speck-and-bacterial-spot-tomatoes', 'description': 'Bacterial speck and spot are common bacterial diseases of tomatoes in the southeastern United States.', 'title': 'Bacterial Speck and Bacterial Spot in Tomatoes | Mississippi State University Extension Service'}, page_content='July 3, 2024 \n Extension specialist recognized for cucurbit virus work  \n\n June 18, 2024 \n Growers look for blueberry rust after it is found in state  \n\nPages1\n2\n3\n4\n5\n6\n7\n…\nnext ›\nlast »\n\n \n\nRelated Publications\n\n\n\n Publica

{'input': 'How did IBM use watsonx at the 2024 US Open Tennis Championship?',
 'history': 'Human: For an apple tree with apple scab, what are the symptoms and how can it be treated?\nAI: Symptoms of apple scab on an apple tree include brown lesions on foliage and fruits, which can enlarge and darken to become more or less circular. Treatment involves cultural management, such as collecting and removing fallen leaves and infected twigs, and sanitation practices like leaf litter removal and pruning, ideally done in the fall or winter before bud break. Resistant cultivars are the best way to prevent apple scab, with many species and cultivars of apple and crabapple being resistant to the scab fungus. Chemical management, such as fungicide sprays applied at bud swell and according to label directions, can also effectively manage apple scab by reducing primary infection.',
 'output': "IBM did not use Watsonx at the 2024 US Open Tennis Championship. The provided context does not contain info

In [25]:
agent_executor.invoke({"input": "What is the capital of France?"})




> Entering new AgentExecutor chain...


AI:
Thought: The human is asking for the capital of France. I do not have this information directly, but I can use the get_IBM_US_Open_context tool to find out if IBM has any involvement with the French Open Tennis Championship.
Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "IBM's involvement in the French Open Tennis Championship"
}
```
Observation[Document(metadata={'title': 'Identify and Control Late Blight', 'language': 'en', 'description': 'Learn how to identify, control and prevent late blight.', 'source': 'https://www.gardentech.com/disease/late-blight'}, page_content='français (Canada)\r\n                        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProducts\n\n \n\n\n\nLearning\n\n \n\n\n\nIdentify Insects\n\n \n\n\n\nIdentify Diseases\n\n \n\n\n\nFAQs\n\n \n\n\n\nTalk To Us\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n \n\n\n\n\n\n\nGardenTech >\n\n\nManage Plant Disease >\n\n\nLate Blig

{'input': 'What is the capital of France?',
 'history': "Human: For an apple tree with apple scab, what are the symptoms and how can it be treated?\nAI: Symptoms of apple scab on an apple tree include brown lesions on foliage and fruits, which can enlarge and darken to become more or less circular. Treatment involves cultural management, such as collecting and removing fallen leaves and infected twigs, and sanitation practices like leaf litter removal and pruning, ideally done in the fall or winter before bud break. Resistant cultivars are the best way to prevent apple scab, with many species and cultivars of apple and crabapple being resistant to the scab fungus. Chemical management, such as fungicide sprays applied at bud swell and according to label directions, can also effectively manage apple scab by reducing primary infection.\nHuman: How did IBM use watsonx at the 2024 US Open Tennis Championship?\nAI: IBM did not use Watsonx at the 2024 US Open Tennis Championship. The provided

In [26]:
agent_executor.invoke({"input": "Where was the 2026 US Open Tennis Championship?"})




> Entering new AgentExecutor chain...


Action:
```
{
  "action": "get_IBM_US_Open_context",
  "action_input": "Where was the 2026 US Open Tennis Championship?"
}
```
Observation[Document(metadata={'description': 'Bacterial speck and spot are common bacterial diseases of tomatoes in the southeastern United States.', 'source': 'http://extension.msstate.edu/publications/bacterial-speck-and-bacterial-spot-tomatoes', 'title': 'Bacterial Speck and Bacterial Spot in Tomatoes | Mississippi State University Extension Service', 'language': 'en'}, page_content='July 3, 2024 \n Extension specialist recognized for cucurbit virus work  \n\n June 18, 2024 \n Growers look for blueberry rust after it is found in state  \n\nPages1\n2\n3\n4\n5\n6\n7\n…\nnext ›\nlast »\n\n \n\nRelated Publications\n\n\n\n Publication Number:  P3524 \n HappyHealthy Fact Sheet: Watermelon  \n\n Publication Number:  P4059 \n Basics of Growing Small Fruit Crops in Mississippi  \n\n Publication Number:  P1779 \n  Fruit and 

{'input': 'Where was the 2026 US Open Tennis Championship?',
 'history': "Human: For an apple tree with apple scab, what are the symptoms and how can it be treated?\nAI: Symptoms of apple scab on an apple tree include brown lesions on foliage and fruits, which can enlarge and darken to become more or less circular. Treatment involves cultural management, such as collecting and removing fallen leaves and infected twigs, and sanitation practices like leaf litter removal and pruning, ideally done in the fall or winter before bud break. Resistant cultivars are the best way to prevent apple scab, with many species and cultivars of apple and crabapple being resistant to the scab fungus. Chemical management, such as fungicide sprays applied at bud swell and according to label directions, can also effectively manage apple scab by reducing primary infection.\nHuman: How did IBM use watsonx at the 2024 US Open Tennis Championship?\nAI: IBM did not use Watsonx at the 2024 US Open Tennis Champions